In [9]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

> 官方文档： https://docs.python.org/3.9/library/queue.html#module-queue

# Queue 对象
队列对象 (Queue, LifoQueue, 或者 PriorityQueue) 提供下列描述的公共方法。

- Queue.qsize()
返回队列的大致大小。注意，qsize() > 0 不保证后续的 get() 不被阻塞，qsize() < maxsize 也不保证 put() 不被阻塞。

- Queue.empty()
如果队列为空，返回 True ，否则返回 False 。如果 empty() 返回 True ，不保证后续调用的 put() 不被阻塞。类似的，如果 empty() 返回 False ，也不保证后续调用的 get() 不被阻塞。

- Queue.put(item, block=True, timeout=None)
将 item 放入队列。

- Queue.get(block=True, timeout=None)
从队列中移除并返回一个项目

- Queue.join()
阻塞至队列中所有的元素都被接收和处理完毕。

- Queue.task_done()
表示前面排队的任务已经被完成。被队列的消费者线程使用。每个 get() 被用于获取一个任务， 后续调用 task_done() 告诉队列，该任务的处理已经完成。

In [21]:
import threading, queue

q = queue.Queue()

def worker():
    while True:
        item = q.get()
        print(f'Working on {item}')
        print(f'Finished {item}')
        q.task_done()

# turn-on the worker thread
threading.Thread(target=worker, daemon=True).start()

# send thirty task requests to the worker
for item in range(5):
    q.put(item)
print('All task requests sent\n', end='')

# block until all tasks are done
q.join()
print('All work completed')

All task requests sent
Working on 0
Finished 0
Working on 1
Finished 1
Working on 2
Finished 2
Working on 3
Finished 3
Working on 4
Finished 4
All work completed


# 双端队列deque
注意，这个是在 collectitons 类中
> https://docs.python.org/3.9/library/collections.html#collections.deque

In [14]:
import collections
# 基本操作
dq = collections.deque()
dq.append(1)
dq.extend([2,3])
dq  # deque([1, 2, 3])
lst = [1,2,3,4]
dq2 = collections.deque(lst)
dq2
# 最重要的两个 2 个操作：队列先进先出，左边弹出复杂度为O(1)
dq2.popleft()
dq2.pop()
dq2.appendleft(66)
dq2.append(88)

deque([1, 2, 3])

deque([1, 2, 3, 4])

1

4

# Python 优先队列 queue.PriorityQueue——美丽的优先级队列

1. 什么是优先级队列？
- 优先级队列是一种容器型数据结构，它能管理一队记录，并按照排序字段（例如一个数字类型的权重值）为其排序。
- 由于是排序的，所以在优先级队列中你可以快速获取到最大的和最小的值。
- 可以认为优先级队列是一种修改过的普通队列：普通队列依据记录插入的时间来获取下一个记录，而优先级队列依据优先级来获取下一个记录，优先级取决于排序字段的值。

优先级队列常用来解决调度问题，比如给紧急的任务更高的优先级。
以操作系统的任务调度为例：高优先级的任务（比如实时游戏）应该先于低优先级的任务（比如后台下载软件更新）执行。

PriorityQueue的实现就是调用heap模块，只是基于Queue类进行了封装，加入了同步机制。

In [2]:
from queue import PriorityQueue

q = PriorityQueue()

q.put((2, 'code'))
q.put((1, 'eat'))
q.put((3, 'sleep'))

while not q.empty():
    next_item = q.get()
    print(next_item)

# 结果：
#   (1, 'eat')
#   (2, 'code')
#   (3, 'sleep')


(1, 'eat')
(2, 'code')
(3, 'sleep')


In [3]:
import queue

q = queue.PriorityQueue()
q.put([1, 'HaiCoder'])  # 1是级别最高的
q.put([40, 1024])
q.put([3, 'Python'])
q.put([5, True])
if __name__ == '__main__':
    print("嗨客网(www.haicoder.net)")
    while not q.empty():  # 不为空时候执行
        print("get data =", q.get())

嗨客网(www.haicoder.net)
get data = [1, 'HaiCoder']
get data = [3, 'Python']
get data = [5, True]
get data = [40, 1024]


当一个对象的所有元素都是可比较的时，默认情况下是根据队列中的对象的第一个元素进行排序，越小的优先级越高，排在越前面。
当第一个元素相同时，依次比较后续的元素的大小来进行排序。

由于 PriorityQueue 是继承自 Queue 类，所以很多函数的用法可以直接参照于 Queue 类中的函数。

In [9]:
from queue import PriorityQueue as PQ

pq = PQ()
pq.put((3, 'a'))
pq.put((2, 'c'))
pq.put((2, 'b'))
pq.put((2, 'b'))

# print(pq.queue) # [(1, 'a'), (2, 'b'), (2, 'b'), (2, 'c')]
# item0 = pq.get() # (1, 'a')
# item0
# print(pq.queue) # [(2, 'b'), (2, 'b'), (2, 'c')]
pq.queue[0]
pq.qsize()

# print(pq.qsize()) # 优先队列的尺寸
#
# while not pq.empty():
#     print(pq.get())

(2, 'b')

4

# 自定义队列的优先级
- Python可以基于queue.PriorityQueue来实现优先队列，但是queue.PriorityQueue的自定义排序不支持函数参数传入，
- 而是只能基于queue.PriorityQueue加入的元素的自身比较器（如__lt__和__gt__）来排序

In [25]:
class Task:
    def __init__(self, taskId, priority, need, arrived):
        """
        Args:
            taskId:任务 id
            priority:任务优先级
            need:任务需要的执行时间
            arrived:任务的到达时间
        """
        self.taskId = taskId
        self.priority = priority
        self.need = need
        self.arrived = arrived

    def __gt__(self, other):
        """
        优先级队列的比较器规则：
        1.优先级数字越大优先级越高
        2.相同优先级的任务选择到达时间最早的任务
        """
        if self.priority != other.priority:
            return other.priority > self.priority
        else:
            return self.arrived > other.arrived


# 测试
import queue

# datas = [list(map(int, input().split())) for _ in range(6)]
datas = [[1, 3, 5, 1], [2, 1, 5, 10], [3, 2, 7, 12], [4, 3, 2, 20], [5, 4, 9, 21], [6, 4, 2, 22]]
pq = queue.PriorityQueue()
for seq in datas:
    tid, p, ptime, atime = seq
    task = Task(tid, p, ptime, atime)
    pq.put(task)

pq.qsize()
while pq.qsize() > 0:
    pollTask = pq.get()
    print(pollTask.taskId)

6

5
6
1
4
3
2
